In [1]:
import sys
sys.path.insert(0,'..')

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set_theme()
import math
from utils.data_utils import get_full_df, get_globals_df

In [3]:
pd.set_option('display.max_columns', 50)
START_POI = 0
END_POI = 20
START_DATE = '2018-12-31'
TOTAL_DAYS = 400
TRAIN_RATIO = 0.7
VALID_RATIO = 0.2
TEST_RATIO = 1 - (TRAIN_RATIO + VALID_RATIO)
WINDOW_SIZE = 24
HORIZON = 6

pred_saved_path = f'/home/users/arash/NGA Project/POI/code/StemGNN/output/houston_data/train_{START_POI}_{END_POI}'
csv_path = '/home/users/arash/datasets/safegraph/weekly_patterns_2018-12-31_2020-06-08_Houston.csv'
poi_info_csv_path = '/home/users/arash/datasets/safegraph/core_poi_info_2018-12-31_2020-06-08.csv'

In [4]:
np.genfromtxt(f'{pred_saved_path}/target.csv', delimiter=',').shape

(1891, 20)

In [5]:
df = get_full_df(csv_path_weekly=csv_path, 
                         poi_info_csv_path=poi_info_csv_path, 
                         start_row=START_POI, end_row=END_POI, 
                         total_days=TOTAL_DAYS)

core_poi-part2.csv
core_poi-part5.csv
core_poi-part4.csv
core_poi-part3.csv
core_poi-part1.csv


In [6]:
df

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.986670,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,1.713640e+10,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [7]:
get_globals_df(df)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032.0,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056.0,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066.0,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.986670,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,1.713640e+10,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024.0,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030.0,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [8]:
df.head(3)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN


In [9]:
df['visits_by_each_hour'] = df['visits_by_each_hour'].apply(lambda x: np.array(x))
df['visits_by_day'] = df['visits_by_day'].apply(lambda x: np.array(x))
agg_df = df.groupby('top_category').agg({'visits_by_day': 'sum',
                                    'visits_by_each_hour': 'sum',
                                    'raw_visit_counts': 'sum'})

In [10]:
agg_df

,visits_by_day,visits_by_each_hour,raw_visit_counts
top_category,,,
Educational Support Services,"[149, 114, 203, 222, 254, 183, 201, 265, 315, ...","[5, 6, 3, 4, 2, 3, 4, 2, 4, 4, 4, 8, 8, 8, 11,...",343710
Elementary and Secondary Schools,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700
Gasoline Stations,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988
General Medical and Surgical Hospitals,"[400, 297, 659, 748, 655, 369, 335, 554, 797, ...","[5, 8, 7, 9, 7, 8, 31, 19, 13, 25, 22, 14, 22,...",352274
Lessors of Real Estate,"[11119, 7918, 8673, 9671, 11360, 11776, 8010, ...","[23, 11, 17, 11, 20, 32, 50, 137, 210, 344, 53...",4202974
"Museums, Historical Sites, and Similar Institutions","[44, 14, 62, 121, 118, 6710, 38, 61, 54, 53, 6...","[0, 1, 1, 0, 1, 0, 1, 2, 6, 8, 5, 5, 2, 5, 2, ...",367989
Spectator Sports,"[26, 33, 51, 58, 74, 29, 18, 40, 69, 70, 68, 9...","[0, 0, 0, 1, 0, 0, 6, 1, 1, 2, 1, 0, 3, 1, 2, ...",357560
Support Activities for Air Transportation,"[13509, 14619, 18100, 16736, 16387, 14310, 173...","[153, 135, 124, 260, 515, 628, 686, 660, 887, ...",7406570
Traveler Accommodation,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310


In [11]:
df_global = df.append(df.agg({'visits_by_day': 'sum',
        'visits_by_each_hour': 'sum',
        'raw_visit_counts': 'sum',
        }), ignore_index=True)

In [12]:
glob_df = df.agg({'visits_by_day': 'sum',
    'visits_by_each_hour': 'sum',
    'raw_visit_counts': 'sum',
    })
glob_df = pd.DataFrame(glob_df).T

In [13]:
glob_df 

,visits_by_day,visits_by_each_hour,raw_visit_counts
0,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075


In [14]:
new_df = pd.DataFrame(np.repeat(glob_df.values, 5, axis=0))
new_df.columns = glob_df.columns
new_df['safegraph_place_id'] = [f'Glob_{i}' for i in range(len(new_df))]
new_df['top_category'] = [f'Glob_{i}' for i in range(len(new_df))]
new_df['sub_category'] = [f'Glob_{i}' for i in range(len(new_df))]
new_df

,visits_by_day,visits_by_each_hour,raw_visit_counts,safegraph_place_id,top_category,sub_category
0,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,Glob_0,Glob_0,Glob_0
1,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,Glob_1,Glob_1,Glob_1
2,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,Glob_2,Glob_2,Glob_2
3,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,Glob_3,Glob_3,Glob_3
4,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,Glob_4,Glob_4,Glob_4


In [15]:
df.append(new_df, ignore_index=True)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032.0,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056.0,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066.0,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.986670,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,1.713640e+10,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024.0,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030.0,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [16]:
df_global.iloc[-1] = df_global.iloc[-1].fillna('Global-0')
df_global

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032.0,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056.0,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066.0,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.98667,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,17136403000.0,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024.0,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030.0,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [17]:
df_global.append(agg_df, ignore_index=True)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032.0,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056.0,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066.0,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.98667,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,17136403000.0,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024.0,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032.0,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030.0,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [18]:
agg_df = df.groupby('sub_category').agg({'visits_by_day': 'sum',
                                    'visits_by_each_hour': 'sum',
                                    'raw_visit_counts': 'sum',
                                    'safegraph_place_id': 'count'})

In [19]:
agg_df

,visits_by_day,visits_by_each_hour,raw_visit_counts,safegraph_place_id
sub_category,,,,
Educational Support Services,"[149, 114, 203, 222, 254, 183, 201, 265, 315, ...","[5, 6, 3, 4, 2, 3, 4, 2, 4, 4, 4, 8, 8, 8, 11,...",343710,1
Elementary and Secondary Schools,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,1
Gasoline Stations with Convenience Stores,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,1
General Medical and Surgical Hospitals,"[400, 297, 659, 748, 655, 369, 335, 554, 797, ...","[5, 8, 7, 9, 7, 8, 31, 19, 13, 25, 22, 14, 22,...",352274,1
Historical Sites,"[44, 14, 62, 121, 118, 6710, 38, 61, 54, 53, 6...","[0, 1, 1, 0, 1, 0, 1, 2, 6, 8, 5, 5, 2, 5, 2, ...",367989,1
Hotels (except Casino Hotels) and Motels,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,1
Malls,"[11119, 7918, 8673, 9671, 11360, 11776, 8010, ...","[23, 11, 17, 11, 20, 32, 50, 137, 210, 344, 53...",4202974,7
Other Airport Operations,"[13509, 14619, 18100, 16736, 16387, 14310, 173...","[153, 135, 124, 260, 515, 628, 686, 660, 887, ...",7406570,6
Sports Teams and Clubs,"[26, 33, 51, 58, 74, 29, 18, 40, 69, 70, 68, 9...","[0, 0, 0, 1, 0, 0, 6, 1, 1, 2, 1, 0, 3, 1, 2, ...",357560,1


In [20]:
agg_df['top_category'] = agg_df.index

In [21]:
agg_df

,visits_by_day,visits_by_each_hour,raw_visit_counts,safegraph_place_id,top_category
sub_category,,,,,
Educational Support Services,"[149, 114, 203, 222, 254, 183, 201, 265, 315, ...","[5, 6, 3, 4, 2, 3, 4, 2, 4, 4, 4, 8, 8, 8, 11,...",343710,1,Educational Support Services
Elementary and Secondary Schools,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,1,Elementary and Secondary Schools
Gasoline Stations with Convenience Stores,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,1,Gasoline Stations with Convenience Stores
General Medical and Surgical Hospitals,"[400, 297, 659, 748, 655, 369, 335, 554, 797, ...","[5, 8, 7, 9, 7, 8, 31, 19, 13, 25, 22, 14, 22,...",352274,1,General Medical and Surgical Hospitals
Historical Sites,"[44, 14, 62, 121, 118, 6710, 38, 61, 54, 53, 6...","[0, 1, 1, 0, 1, 0, 1, 2, 6, 8, 5, 5, 2, 5, 2, ...",367989,1,Historical Sites
Hotels (except Casino Hotels) and Motels,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,1,Hotels (except Casino Hotels) and Motels
Malls,"[11119, 7918, 8673, 9671, 11360, 11776, 8010, ...","[23, 11, 17, 11, 20, 32, 50, 137, 210, 344, 53...",4202974,7,Malls
Other Airport Operations,"[13509, 14619, 18100, 16736, 16387, 14310, 173...","[153, 135, 124, 260, 515, 628, 686, 660, 887, ...",7406570,6,Other Airport Operations
Sports Teams and Clubs,"[26, 33, 51, 58, 74, 29, 18, 40, 69, 70, 68, 9...","[0, 0, 0, 1, 0, 0, 6, 1, 1, 2, 1, 0, 3, 1, 2, ...",357560,1,Sports Teams and Clubs


### Global DF by types

In [22]:
df

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,"[6428, 6911, 8643, 8195, 7936, 7265, 8236, 614...","[74, 42, 23, 123, 220, 284, 338, 311, 493, 331...",3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032,US,NaN,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,"[3688, 3007, 3308, 3478, 3947, 4036, 2419, 212...","[8, 1, 4, 3, 5, 12, 18, 50, 61, 102, 158, 239,...",1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056,US,SG_BRAND_0a3c99595c9d3fddfece9c4e7607e5b3,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN
3,sg:73e44ffdbcb24363bf01b6158373b9f3,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,George Bush Intercontinental Airport Terminal E,3950 S Terminal Terminal E,Houston,TX,77066,US,NaN,NaN,sg:7dea7a58e8424b22ba0d0b96bc1b6cc9,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110.0,29.986670,-95.337942,NaN,NaN,NaN
4,sg:360d88ef2ced4be180ea24290b9f9df4,"[2410, 2607, 3235, 2913, 2806, 2217, 3011, 217...","[28, 45, 53, 36, 94, 116, 104, 132, 103, 189, ...",1240632,William P Hobby Airport,7800 Airport Blvd,Houston,TX,77061,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.646145,-95.277014,1.713640e+10,NaN,NaN
5,sg:97332bde5ea043d6ac3cfa2fdfe63403,"[2771, 2348, 2462, 2606, 2995, 3433, 2037, 146...","[4, 2, 0, 1, 1, 9, 7, 31, 45, 68, 152, 265, 34...",975482,Bally Fitness Center Shopping Center,9801 Katy Fwy,Houston,TX,77024,US,NaN,NaN,NaN,Lessors of Real Estate,Malls,531120.0,29.781627,-95.540084,NaN,NaN,NaN
6,sg:b7503ea7b2d9441f8a8f3c34c970ea44,"[1760, 2036, 2434, 2120, 2047, 1624, 2287, 146...","[13, 35, 37, 22, 77, 87, 82, 99, 75, 143, 89, ...",913738,William P Hobby Airport Terminal 2,7800 Airport Blvd,Houston,TX,77061,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.653759,-95.276671,NaN,NaN,NaN
7,sg:535230c8e0994e19ba40007c69d4c5c2,"[1099, 966, 1288, 1242, 1275, 1068, 1266, 997,...","[20, 10, 5, 54, 81, 48, 48, 20, 50, 72, 65, 57...",588942,George Bush Intercontinental Airport Terminal A,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.987013,-95.349958,NaN,NaN,NaN
8,sg:e6b1d9d411f244d2bd0d48c64dce5c57,"[944, 1075, 1324, 1160, 1227, 1064, 1367, 871,...","[4, 2, 4, 21, 25, 74, 67, 62, 48, 46, 55, 37, ...",551983,George Bush Intercontinental Airport Terminal E,2800 N Terminal Rd,Houston,TX,77032,US,NaN,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.985676,-95.334277,NaN,NaN,NaN
9,sg:09be850d3529490f835265b55785d548,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Pediatric Education And Creative Arts Program,1515 Holcombe Blvd Unit 87,Houston,TX,77030,US,NaN,NaN,NaN,Elementary and Secondary Schools,Elementary and Secondary Schools,611110.0,29.707119,-95.396997,NaN,NaN,NaN


In [26]:
res_df = df.copy()
res_df['visits_by_each_hour'] = df['visits_by_each_hour'].apply(lambda x: np.array(x))
res_df['visits_by_day'] = df['visits_by_day'].apply(lambda x: np.array(x))
glob_df = res_df.groupby('top_category').agg({'visits_by_day': 'sum',
'visits_by_each_hour': 'sum',
'raw_visit_counts': 'sum',
})
glob_df = pd.DataFrame(glob_df)
glob_df['top_category'] = glob_df.index
glob_df['sub_category'] = ['Global' for i in range(len(glob_df))]
glob_df['safegraph_place_id'] = [f'Category_Global_{i}' for i in range(len(glob_df))]
print(glob_df.shape)
glob_df

(9, 6)


,visits_by_day,visits_by_each_hour,raw_visit_counts,top_category,sub_category,safegraph_place_id
top_category,,,,,,
Educational Support Services,"[149, 114, 203, 222, 254, 183, 201, 265, 315, ...","[5, 6, 3, 4, 2, 3, 4, 2, 4, 4, 4, 8, 8, 8, 11,...",343710,Educational Support Services,Global,Category_Global_0
Elementary and Secondary Schools,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,Elementary and Secondary Schools,Global,Category_Global_1
Gasoline Stations,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,Gasoline Stations,Global,Category_Global_2
General Medical and Surgical Hospitals,"[400, 297, 659, 748, 655, 369, 335, 554, 797, ...","[5, 8, 7, 9, 7, 8, 31, 19, 13, 25, 22, 14, 22,...",352274,General Medical and Surgical Hospitals,Global,Category_Global_3
Lessors of Real Estate,"[11119, 7918, 8673, 9671, 11360, 11776, 8010, ...","[23, 11, 17, 11, 20, 32, 50, 137, 210, 344, 53...",4202974,Lessors of Real Estate,Global,Category_Global_4
"Museums, Historical Sites, and Similar Institutions","[44, 14, 62, 121, 118, 6710, 38, 61, 54, 53, 6...","[0, 1, 1, 0, 1, 0, 1, 2, 6, 8, 5, 5, 2, 5, 2, ...",367989,"Museums, Historical Sites, and Similar Institu...",Global,Category_Global_5
Spectator Sports,"[26, 33, 51, 58, 74, 29, 18, 40, 69, 70, 68, 9...","[0, 0, 0, 1, 0, 0, 6, 1, 1, 2, 1, 0, 3, 1, 2, ...",357560,Spectator Sports,Global,Category_Global_6
Support Activities for Air Transportation,"[13509, 14619, 18100, 16736, 16387, 14310, 173...","[153, 135, 124, 260, 515, 628, 686, 660, 887, ...",7406570,Support Activities for Air Transportation,Global,Category_Global_7
Traveler Accommodation,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,Traveler Accommodation,Global,Category_Global_8


In [28]:
new_df = get_globals_df(df, 5*math.ceil(glob_df.shape[0]/5) - glob_df.shape[0])
new_df.append(glob_df, ignore_index=True).tail(10)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
20,Glob_0,"[31174, 29244, 35926, 35316, 36309, 39914, 329...","[234, 196, 178, 351, 689, 892, 1116, 1184, 159...",16380075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Global,Global,NaN,NaN,NaN,NaN,NaN,NaN
21,Category_Global_0,"[149, 114, 203, 222, 254, 183, 201, 265, 315, ...","[5, 6, 3, 4, 2, 3, 4, 2, 4, 4, 4, 8, 8, 8, 11,...",343710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Educational Support Services,Global,NaN,NaN,NaN,NaN,NaN,NaN
22,Category_Global_1,"[767, 321, 1035, 1202, 1218, 461, 406, 962, 13...","[6, 7, 10, 1, 5, 16, 68, 79, 70, 67, 66, 62, 3...",549700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elementary and Secondary Schools,Global,NaN,NaN,NaN,NaN,NaN,NaN
23,Category_Global_2,"[2681, 3062, 3678, 3380, 3220, 3136, 3435, 255...","[23, 16, 8, 34, 72, 107, 143, 148, 212, 112, 1...",1448988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gasoline Stations,Global,NaN,NaN,NaN,NaN,NaN,NaN
24,Category_Global_3,"[400, 297, 659, 748, 655, 369, 335, 554, 797, ...","[5, 8, 7, 9, 7, 8, 31, 19, 13, 25, 22, 14, 22,...",352274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,General Medical and Surgical Hospitals,Global,NaN,NaN,NaN,NaN,NaN,NaN
25,Category_Global_4,"[11119, 7918, 8673, 9671, 11360, 11776, 8010, ...","[23, 11, 17, 11, 20, 32, 50, 137, 210, 344, 53...",4202974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lessors of Real Estate,Global,NaN,NaN,NaN,NaN,NaN,NaN
26,Category_Global_5,"[44, 14, 62, 121, 118, 6710, 38, 61, 54, 53, 6...","[0, 1, 1, 0, 1, 0, 1, 2, 6, 8, 5, 5, 2, 5, 2, ...",367989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Global,NaN,NaN,NaN,NaN,NaN,NaN
27,Category_Global_6,"[26, 33, 51, 58, 74, 29, 18, 40, 69, 70, 68, 9...","[0, 0, 0, 1, 0, 0, 6, 1, 1, 2, 1, 0, 3, 1, 2, ...",357560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spectator Sports,Global,NaN,NaN,NaN,NaN,NaN,NaN
28,Category_Global_7,"[13509, 14619, 18100, 16736, 16387, 14310, 173...","[153, 135, 124, 260, 515, 628, 686, 660, 887, ...",7406570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Activities for Air Transportation,Global,NaN,NaN,NaN,NaN,NaN,NaN
29,Category_Global_8,"[2479, 2866, 3465, 3178, 3023, 2940, 3209, 240...","[19, 12, 8, 31, 67, 98, 127, 136, 192, 108, 16...",1350310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Traveler Accommodation,Global,NaN,NaN,NaN,NaN,NaN,NaN
